<a href="https://colab.research.google.com/github/fourfeatherz/DS2002F24/blob/main/Python_API_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Requesting Data from Application Programming Interfaces (API's)
This notebook demonstrates the fundamentals of interacting with a web-hosted API for the sake of data retrieval. Much of this functionality is made available through the **requests** library which should have already been installed on your machine as part of the **Anaconda** python distribution. Documentation for the **requests** library is here:
https://docs.python-requests.org/en/latest/user/quickstart/.

### 1.0. Prerequisites
If you find that the **requests**  library isn't already installed on your machine then simply run the following command in a new **Terminal** window in your Jupyter environment... just as you have in following labs.
-  python -m pip install requests

#### 1.1. Import the libaries that you'll be working with in the notebook

In [1]:
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd

### 2.0. Issue a Request to an API Endpoint
The following function issues a **request** to a REST API endpoint via the HTTP request/response mechanism. It demonstrates returning the *JSON payload* of the **response** object as one of two **response_types**; either as a **string** or as a **Pandas DataFrame**.  

#### 2.1. Exception Handling:
In order to cope with the stateless nature of HTTP communications, the **get_api_response()** function implements extensive **exception handling**. When attempting to connect to an HTTP endpoint, the following response **status_codes** may be returned:
- **200:** Everything went okay, and the result has been returned (if any).
- **301:** The server is redirecting you to a different endpoint. This can happen when a company switches domain names, or an endpoint name is changed.
- **400:** The server thinks you made a bad request. This can happen when you don’t send along the right data, among other things.
- **401:** The server thinks you’re not authenticated. Many APIs require login ccredentials, so this happens when you submit the wrong credentials.
- **403:** The resource you’re trying to access is forbidden: you don’t have the right perlessons to see it.
- **404:** The resource you tried to access wasn’t found on the server.
- **503:** The server is not ready to handle the request.

In [2]:
def get_api_response(url, response_type):
    try:
        response = requests.get(url)
        response.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        result = json.dumps(response.json(), sort_keys=True, indent=4)
    elif response_type == 'dataframe':
        result = pd.json_normalize(response.json())
    else:
        result = "An unhandled error has occurred!"

    return result

#### 2.2. Unit test to ensure proper exception handling functionality

In [3]:
bad_url = "https://api.open-notify.org/this-api-doesnt-exist"
valid_url = "http://universities.hipolabs.com/search?name=middle"

response_type = ['json', 'dataframe']

In [4]:
json_string = get_api_response(bad_url, response_type[0])
print(json_string)

An Error Connecting to the API occurred: ConnectionError(MaxRetryError("HTTPSConnectionPool(host='api.open-notify.org', port=443): Max retries exceeded with url: /this-api-doesnt-exist (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x795e070321a0>: Failed to establish a new connection: [Errno 111] Connection refused'))"))


In [5]:
df = get_api_response(bad_url, response_type[1])
print(df)

An Error Connecting to the API occurred: ConnectionError(MaxRetryError("HTTPSConnectionPool(host='api.open-notify.org', port=443): Max retries exceeded with url: /this-api-doesnt-exist (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x795e070335e0>: Failed to establish a new connection: [Errno 111] Connection refused'))"))


#### 2.3. Unit test to ensure proper data retrieval functionality
Here we can see that when specifying **response_type[0]** we get back a **string in JSON format**, and when specifying **response_type[1]** we get back a **Pandas DataFrame**.  On closer inspection we can observe that the JSON payload is in the form of a **list** of **dictionaries**, each of which includes nested **lists** for the **domains** and **web_pages** fields in addition to the other fields that are formatted in simple **"key" : "value"** format. This presents a problem we will have to handle in order to have a correctly formed **DataFrame** because, as we learned when desiging **OLTP** databases, having multiple values in a single column violates the **First Normal Form**.

In [6]:
json_string = get_api_response(valid_url, response_type[0])
print(json_string)

[
    {
        "alpha_two_code": "KW",
        "country": "Kuwait",
        "domains": [
            "aum.edu.kw"
        ],
        "name": "American University of Middle East",
        "state-province": null,
        "web_pages": [
            "http://www.aum.edu.kw/"
        ]
    },
    {
        "alpha_two_code": "US",
        "country": "United States",
        "domains": [
            "middlebury.edu"
        ],
        "name": "Middlebury College",
        "state-province": null,
        "web_pages": [
            "http://www.middlebury.edu/"
        ]
    },
    {
        "alpha_two_code": "US",
        "country": "United States",
        "domains": [
            "mtsu.edu"
        ],
        "name": "Middle Tennessee State University",
        "state-province": null,
        "web_pages": [
            "http://www.mtsu.edu/"
        ]
    },
    {
        "alpha_two_code": "TR",
        "country": "Turkey",
        "domains": [
            "metu.edu.tr"
        ],
        "na

In [7]:
df = get_api_response(valid_url, response_type[1])

print(df.shape)
print(df.columns)

df.info()

(10, 6)
Index(['domains', 'alpha_two_code', 'web_pages', 'name', 'state-province',
       'country'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   domains         10 non-null     object
 1   alpha_two_code  10 non-null     object
 2   web_pages       10 non-null     object
 3   name            10 non-null     object
 4   state-province  0 non-null      object
 5   country         10 non-null     object
dtypes: object(6)
memory usage: 608.0+ bytes


In [8]:
df

,domains,alpha_two_code,web_pages,name,state-province,country
0,[aum.edu.kw],KW,[http://www.aum.edu.kw/],American University of Middle East,None,Kuwait
1,[middlebury.edu],US,[http://www.middlebury.edu/],Middlebury College,None,United States
2,[mtsu.edu],US,[http://www.mtsu.edu/],Middle Tennessee State University,None,United States
3,[metu.edu.tr],TR,[http://www.metu.edu.tr/],Middle East Technical University,None,Turkey
4,[meu.edu.jo],JO,[http://www.meu.edu.jo/],Middle East University,None,Jordan
5,[mga.edu],US,[http://www.mga.edu/],Middle Georgia State College,None,United States
6,[mdx.ac.uk],GB,[https://www.mdx.ac.uk/],Middlesex University - London,None,United Kingdom
7,[middlesex.mass.edu],US,[http://www.middlesex.mass.edu],Middlesex Community College,None,United States
8,[middlesexcc.edu],US,[http://www.middlesexcc.edu],Middlesex County College,None,United States
9,"[middlesbro.ac.uk, mbro.ac.uk]",GB,[https://www.mbro.ac.uk/],Middlesbrough College,None,United Kingdom


#### 2.3. Perform Desired Transformations
In any ETL process, there will be some form of data **transformation**.  Here we will explore transforming JSON data.

As identified above, the first issue we must handle is the nested **lists** that may contain multiple **domains** and **web_pages**. To do so we will exploring the advanced capabilities of the Pandas **json_normalize()** function, but first we will create a simplified function that retrieves a JSON object from an API.

In [9]:
def get_api_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    return response.json()

In [11]:
json_data = get_api_data(valid_url)
print(json_data)

[{'domains': ['aum.edu.kw'], 'alpha_two_code': 'KW', 'web_pages': ['http://www.aum.edu.kw/'], 'name': 'American University of Middle East', 'state-province': None, 'country': 'Kuwait'}, {'domains': ['middlebury.edu'], 'alpha_two_code': 'US', 'web_pages': ['http://www.middlebury.edu/'], 'name': 'Middlebury College', 'state-province': None, 'country': 'United States'}, {'domains': ['mtsu.edu'], 'alpha_two_code': 'US', 'web_pages': ['http://www.mtsu.edu/'], 'name': 'Middle Tennessee State University', 'state-province': None, 'country': 'United States'}, {'domains': ['metu.edu.tr'], 'alpha_two_code': 'TR', 'web_pages': ['http://www.metu.edu.tr/'], 'name': 'Middle East Technical University', 'state-province': None, 'country': 'Turkey'}, {'domains': ['meu.edu.jo'], 'alpha_two_code': 'JO', 'web_pages': ['http://www.meu.edu.jo/'], 'name': 'Middle East University', 'state-province': None, 'country': 'Jordan'}, {'domains': ['mga.edu'], 'alpha_two_code': 'US', 'web_pages': ['http://www.mga.edu/']

Next, we can **flatten** (aka, Normalize) the fields containing the nested lists (**domains** and **web_pages**) using the **record_path** parameter of the **pandas.json_normalize** function.

In [12]:
pd.json_normalize(json_data, record_path=['domains'])

,0
0,aum.edu.kw
1,middlebury.edu
2,mtsu.edu
3,metu.edu.tr
4,meu.edu.jo
5,mga.edu
6,mdx.ac.uk
7,middlesex.mass.edu
8,middlesexcc.edu
9,middlesbro.ac.uk


We can confirm that the *domains* field has been flattened since we now have 10 observations where before we had only 9. However, we also want to include other fields; which we accomplish with the **meta** parameter. Note that we've also omitted the **state-province** field since it doesn't appear to contain any useful data. What's more, since it's possible for some **keys** to be missing in a JSON document, we can supress any errors using the **errors='ignore'** parameter.

In [ ]:
df = pd.json_normalize(json_data,
                       record_path=['domains'],
                       meta=['country', 'name', 'alpha_two_code'],
                       errors='ignore')
df

,0,country,name,alpha_two_code
0,middlesbro.ac.uk,United Kingdom,Middlesbrough College,GB
1,mbro.ac.uk,United Kingdom,Middlesbrough College,GB
2,middlesex.mass.edu,United States,Middlesex Community College,US
3,middlesexcc.edu,United States,Middlesex County College,US
4,mdx.ac.uk,United Kingdom,Middlesex University,GB
5,middlebury.edu,United States,Middlebury College,US
6,mga.edu,United States,Middle Georgia State College,US
7,meu.edu.jo,Jordan,Middle East University,JO
8,metu.edu.tr,Turkey,Middle East Technical University,TR
9,mtsu.edu,United States,Middle Tennessee State University,US


Next, we can normalize the **web_pages** list to ensure an unique row for each of its unique values as we add it to the DataFrame.

In [ ]:
df['web_pages'] = pd.json_normalize(json_data, record_path=['web_pages'])
df

,0,country,name,alpha_two_code,web_pages
0,middlesbro.ac.uk,United Kingdom,Middlesbrough College,GB,https://www.mbro.ac.uk/
1,mbro.ac.uk,United Kingdom,Middlesbrough College,GB,http://www.middlesex.mass.edu
2,middlesex.mass.edu,United States,Middlesex Community College,US,http://www.middlesexcc.edu
3,middlesexcc.edu,United States,Middlesex County College,US,http://www.mdx.ac.uk/
4,mdx.ac.uk,United Kingdom,Middlesex University,GB,http://www.middlebury.edu/
5,middlebury.edu,United States,Middlebury College,US,http://www.mga.edu/
6,mga.edu,United States,Middle Georgia State College,US,http://www.meu.edu.jo/
7,meu.edu.jo,Jordan,Middle East University,JO,http://www.metu.edu.tr/
8,metu.edu.tr,Turkey,Middle East Technical University,TR,http://www.mtsu.edu/
9,mtsu.edu,United States,Middle Tennessee State University,US,http://www.aum.edu.kw/


Finally, we create a dictionary to **map** new column names to the old ones using the **rename()** function of the **pandas.DataFrame**.  We also demonstrate how columns can be reordered by simply passing a **list** of column names in the desired order.

In [ ]:
column_name_map = {0 : "Domain",
                   "country" : "Country",
                   "name" : "Institution_Name",
                   "alpha_two_code" : "Country_Code",
                   "web_pages" : "Web_Address"
                  }

df.rename(columns=column_name_map, inplace=True)
df = df[['Institution_Name','Country','Country_Code','Domain','Web_Address']]
df

,Institution_Name,Country,Country_Code,Domain,Web_Address
0,Middlesbrough College,United Kingdom,GB,middlesbro.ac.uk,https://www.mbro.ac.uk/
1,Middlesbrough College,United Kingdom,GB,mbro.ac.uk,http://www.middlesex.mass.edu
2,Middlesex Community College,United States,US,middlesex.mass.edu,http://www.middlesexcc.edu
3,Middlesex County College,United States,US,middlesexcc.edu,http://www.mdx.ac.uk/
4,Middlesex University,United Kingdom,GB,mdx.ac.uk,http://www.middlebury.edu/
5,Middlebury College,United States,US,middlebury.edu,http://www.mga.edu/
6,Middle Georgia State College,United States,US,mga.edu,http://www.meu.edu.jo/
7,Middle East University,Jordan,JO,meu.edu.jo,http://www.metu.edu.tr/
8,Middle East Technical University,Turkey,TR,metu.edu.tr,http://www.mtsu.edu/
9,Middle Tennessee State University,United States,US,mtsu.edu,http://www.aum.edu.kw/


With the data having been **extracted** from an API, and any desired **transformations** having been accomplished, we can now **load** the data into any desired destination; e.g., SQL database, NoSQL database, or data lake (file system).

### 3.0. API Endpoint Authentication & Parameters

In [ ]:
def get_api_response(url, headers, params):
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.InvalidHeader as erri:
        return "A Header Error occurred: " + repr(erri)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    return response.json()

Before you can start using the GitHub API, you'll need to generate a *personal access token* to enable you to authenticate to the API. To create your own token, navigate to https://github.com/settings/tokens and then click the **Generate New Token** button.